In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Cost Supply Network Optimization

### Problem Statement
###### Come up with a best cost supply network (Which supplier which location should service which plants and what qty). It must optimize all constraints, satisfy the full demand at our locations.


**Optimization Function**: Delivered cost = Base Cost + Freight Cost + Any taxes/duties
##### Some additional Points to be taken care of
* If BC Supply <19000 tons, then we incur a penalty of $125/MT
* Supplies to China from outside of South East Asia, incurs an import duty of 15%
* Suppliers to "AP - China" from outside of South East Asia, incurs an import duty of 18%
* If EU is supplied from Asia based suppliers, there is an import duty of 8%.


#### The first step is to open all data sources

##### base_cost

In [2]:
base_cost = pd.read_csv('../data/raw/base_cost.csv', sep=';', decimal=',', encoding='UTF-8')

In [3]:
base_cost

,Location,Cost ($/MT)
0,BC San Pedro,3496
1,BC Malaysia,3942
2,BC Pasir Gudang,3942
3,Olam Singapore,3836
4,Olam Holland,3278


This dataframe will be converted to a numpy array and transposed to ease the calculations

In [4]:
base_cost=np.array((base_cost.values[:,1]),ndmin=2)

In [5]:
base_cost

array([[3496, 3942, 3942, 3836, 3278]], dtype=object)

##### demand

In [6]:
demand=pd.read_csv('../data/raw/demand.csv', sep=';', decimal=',', encoding='UTF-8')

In [7]:
demand

,destiny,quantity
0,East Coast BB,4080
1,East Coast SB,1121
2,West Coast BB,624
3,Salinas BB,5585
4,EU,4607
5,EEMEA,0
6,LA,1703
7,AP - China,3571
8,China,5345


##### freight_matrix

In [8]:
freight_matrix=pd.read_csv('../data/raw/freight_matrix.csv', sep=';', decimal=',', encoding='UTF-8')

In [9]:
freight_matrix

,Unnamed: 0,BC San Pedro,BC Malaysia,BC Pasir Gudang,ADM Singapore,ADM Amsterdam
0,East Coast BB,111,240,240,140,120
1,East Coast SB,111,240,240,140,120
2,West Coast BB,511,240,240,200,120
3,Salinas BB,620,445,445,350,300
4,EU,75,150,150,150,50
5,EEMEA,163,41,41,41,62
6,LA,120,110,110,250,80
7,AP - China,250,60,60,60,250
8,China,300,60,60,60,350


In [10]:
freight_matrix.rename(columns={'Unnamed: 0':''}, inplace=True)
freight_matrix.set_index('',inplace=True)

In [11]:
freight_matrix

,BC San Pedro,BC Malaysia,BC Pasir Gudang,ADM Singapore,ADM Amsterdam
,,,,,
East Coast BB,111,240,240,140,120
East Coast SB,111,240,240,140,120
West Coast BB,511,240,240,200,120
Salinas BB,620,445,445,350,300
EU,75,150,150,150,50
EEMEA,163,41,41,41,62
LA,120,110,110,250,80
AP - China,250,60,60,60,250
China,300,60,60,60,350


##### supply_location_capacity

In [12]:
supply_location_capacity=pd.read_csv('../data/raw/supply_location_capacity.csv', sep=';', decimal=',', encoding='UTF-8')

In [13]:
supply_location_capacity

,Supplier,Location,Max Capacity
0,BC,BC San Pedro,12480
1,BC,BC Malaysia,7200
2,BC,BC Pasir Gudang,7200
3,Olam,Olam Singapore,7500
4,Olam,Olam Holland,6000


##### quantity
This matrix will be started with 0 but the zeros will be optimized 

In [14]:
quantity=np.zeros((9,5))
quantity

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### Support functions

Let´s start defining how the freight_cost is calculated

In [15]:
def matrix_freight_cost_calculator(quantity, freight_matrix):
    return np.reshape(quantity, (-1, 5))*freight_matrix

In [16]:
def total_freight_cost_calculator(matrix):
    return np.sum(matrix.values)

In [17]:
def matrix_base_cost_calculator(quantity,base_cost):
    return np.reshape(quantity, (-1, 5))*base_cost

In [18]:
def total_base_cost_calculator(matrix):      
    return np.sum(matrix)

### Conditions (Taxes/Duties)

If BC Supply <19000 tons, then we incur a penalty of $125/MT

In [19]:
def bc_supply_penalty(quantity):
    # BC supplyers are the first 3 columns of the matrix
    # We will sum them and apply the above rule if the result is < 19000
    bc_quantity= np.sum(np.reshape(quantity, (-1, 5))[:,:3])
    return bc_quantity*125 if bc_quantity <19000 else 0

Supplies to China from outside of South East Asia, incurs an import duty of 15%

In [20]:
def china_tax(calculated_freight_matrix,base_cost_matrix):
    # The only supplier outside South East Asia is ADM Amsterdam
    # The tax will be calculated over the freight cost + base cost for that supplier to Chine
    # The position of this supplyer to China on the calculated_freigh_matrix and base_cost_matrix is 8,4 (9th row, 5th column)
    return ((calculated_freight_matrix.values[8,4])+(base_cost_matrix[8,4]))*0.15

Suppliers to "AP - China" from outside of South East Asia, incurs an import duty of 18%

In [21]:
def ap_china_tax(calculated_freight_matrix,base_cost_matrix):
    # The only supplier outside South East Asia is ADM Amsterdam
    # The tax will be calculated over the freight cost + base cost for that supplier to Chine
    # The position of this supplyer to AP-China on the calculated_freigh_matrix and base_cost_matrix is 7,4 (8th row, 5th column)
    return (calculated_freight_matrix.values[7,4]+base_cost_matrix[7,4])*0.18

If EU is supplied from Asia based suppliers, there is an import duty of 8%.

In [22]:
def eu_asia_tax(calculated_freight_matrix,base_cost_matrix):
    # EU is in row 4 of the matrix (5th line)
    # Asia based suppliers are columns 0 to 3 (4 columns)
    return np.sum((calculated_freight_matrix.values[4,:3]+base_cost_matrix[4,:3]))*0.08

### Time to sum all taxes

In [23]:
def all_taxes_calculator(quantity,calculated_freight_matrix,base_cost_matrix):
    results=[]
    results.append(bc_supply_penalty(quantity))
    results.append(china_tax(calculated_freight_matrix,base_cost_matrix))
    results.append(ap_china_tax(calculated_freight_matrix,base_cost_matrix))
    results.append(eu_asia_tax(calculated_freight_matrix,base_cost_matrix))
    return np.sum(results)

### Constraints

#### Demand Constraints

In [24]:
def constraint1(quantity):
    q = np.reshape(quantity, (-1, 5))    
    return q.sum(axis=1)[0]-4080

In [25]:
def constraint2(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[1]-1121

In [26]:
def constraint3(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[2]-624

In [27]:
def constraint4(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[3]-5585

In [28]:
def constraint5(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[4]-4607

In [29]:
def constraint6(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[5]-0

In [30]:
def constraint7(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[6]-1703

In [31]:
def constraint8(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[7]-3571

In [32]:
def constraint9(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[8]-5345

#### Capacity Constraints

In [33]:
def constraint10(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 12480-q.sum(axis=0)[0]

In [34]:
def constraint11(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 7200-q.sum(axis=0)[1]

In [35]:
def constraint12(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 7200-q.sum(axis=0)[2]

In [36]:
def constraint13(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 7500-q.sum(axis=0)[3]

In [37]:
def constraint14(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 6000-q.sum(axis=0)[4]

In [38]:
con1 = {'type':'ineq', 'fun':constraint1}
con2 = {'type':'ineq', 'fun':constraint2}
con3 = {'type':'ineq', 'fun':constraint3}
con4 = {'type':'ineq', 'fun':constraint4}
con5 = {'type':'ineq', 'fun':constraint5}
con6 = {'type':'ineq', 'fun':constraint6}
con7 = {'type':'ineq', 'fun':constraint7}
con8 = {'type':'ineq', 'fun':constraint8}
con9 = {'type':'ineq', 'fun':constraint9}
con10 = {'type':'ineq', 'fun':constraint10}
con11 = {'type':'ineq', 'fun':constraint11}
con12 = {'type':'ineq', 'fun':constraint12}
con13 = {'type':'ineq', 'fun':constraint13}
con14 = {'type':'ineq', 'fun':constraint14}

In [39]:
cons=[con1,con2,con3,con4,con5,con6,con7,
     con8,con9,con10,con11,con12,con13,con14]

In [40]:
b=(0., None)
bnds=(b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b)

### Optimization Function

The main objective of this code is to minimize the below function

In [41]:
def objective_calculator(quantity,freight_matrix,base_cost):
    taxes = all_taxes_calculator(quantity,
                             matrix_freight_cost_calculator(quantity, freight_matrix),
                             matrix_base_cost_calculator(quantity,base_cost))
    base = total_base_cost_calculator(matrix_base_cost_calculator(quantity,base_cost))
    freight = total_freight_cost_calculator(matrix_freight_cost_calculator(quantity, freight_matrix))
    return base+freight+taxes

In [64]:
min=minimize(objective_calculator,quantity,method='SLSQP', constraints=cons ,bounds=bnds,args=(freight_matrix,base_cost),
            options={'maxiter': 1000000000})

In [65]:
min.success

False

In [46]:
import locale
locale.setlocale( locale.LC_ALL, '' )

'Portuguese_Brazil.1252'

In [47]:
locale.currency( min.fun, grouping=True )

'R$ 100.993.832,06'

In [66]:
pd.DataFrame(np.reshape(min['x'],(-1,5)))

,0,1,2,3,4
0,4079.998910,0.000000,0.000000,0.000000,0.000000
1,1120.998910,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,623.999008
3,0.000000,0.000000,0.000000,4815.998777,768.998955
4,0.000000,0.000000,0.000000,0.000000,4606.999028
5,0.000000,0.000000,0.000000,0.000000,0.000000
6,1702.998907,0.000000,0.000000,0.000000,0.000000
7,3570.998869,0.000000,0.000000,0.000000,0.000000
8,2004.998855,327.998795,327.998795,2683.998862,0.000000
